<h1>Image Exporter for Google Earth Engine</h1>

The code below is the first version of the image exporter. It takes a KML file that's imported from google earth pro, and transforms the polygon in the file into an image that is then exported to google drive

<h2>Imports</h2>

In [ ]:
import ee
import geemap
import geopandas as gpd
from shapely.ops import transform

<h2>Authenticating and Initializing Google Earth Engine</h2>

In [ ]:
# Initialize
try:
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine initialized successfully.")
except Exception as e:
    print(f"Initialization failed: {e}")
    ee.Authenticate()
    ee.Initialize(project='golf-mapper-463720')

<h2>Data Cleaning, File Conversion</h2>
By default, the KML file from Earth Pro has a Z-axis, which is unable to be interpreted by Earth Engine. The code below removes the Z-axis component from the file. After the removal, the code converts the KML file into an Earth Engine Object. Lastly, the geometry is converted into.a 'Geometry' type variable, which allows it to be passed to the image clipping and export functions. 

NOTE: the image clipper can use the 'geometry variable', but later on when we export to drive it needs to be a 'Geometry' type variable, which is called 'my_region' in this cell

In [ ]:
# Read your KML
#gdf = geopandas data frame
#gdf_to_ee() is a built-in earthengine function
gdf = gpd.read_file('Path/to/kml_file') ### <=== EDIT THIS LINE

# Function to remove Z coordinates
def remove_z_dimension(geom):
    if geom.has_z:
        return transform(lambda x, y, z=None: (x, y), geom)
    return geom

# Apply to all geometries
gdf['geometry'] = gdf.geometry.apply(remove_z_dimension)

# Verify it worked
print("After Z removal:")
print("Geometry types:", gdf.geometry.type.value_counts())
print("Sample geometry:", gdf.geometry.iloc[0])

# Now try the Earth Engine conversion
geometry = geemap.gdf_to_ee(gdf)
print("Success! Converted to Earth Engine geometry")
print(f"GEOMETRY TYPE: {type(geometry)}")
print('===============')

print('Converting geometry type...')
my_region = geometry.geometry()
print(f"CONVERTED REGION TYPE: {type(my_region)} ✅")
print(f"CONVERTED REGION INFO: {my_region.getInfo()}")
print('===============')

# We will pass the 'my_region' variable to the exporter function later on

<h2>Clipping the Satellite Image of the Region</h2>

Using images from the sentinel satellite

In [ ]:
# Specifying the start and end date of the satellite imagery
start_date = '2024-01-01'
end_date = '2024-12-31'

# Get the cleanest Sentinel-2 imagery possible
sentinel_clean = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3)) \
    .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 3)) \
    .median()

# Select RGB bands and apply proper scaling
rgb_image = sentinel_clean.select(['B4', 'B3', 'B2']).multiply(0.0001)
clipped_image = rgb_image.clip(geometry)

print("Image Clipped ✅")

<h2>Exporting the image to Google Drive</h2>

In [ ]:
# Enhanced visualization parameters for golf courses
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.02,    # Slightly higher min for better contrast
    'max': 0.28,    # Adjusted max for golf course greens
    'gamma': 1.6    # Higher gamma for more vibrant colors
}

vis_image = clipped_image.visualize(**vis_params)

# Export with optimal settings
task = ee.batch.Export.image.toDrive(
    image=vis_image,
    description='Description of the Clipped image', ### <=== EDIT THIS LINE
    folder='My_googledrive_folder', ### <=== EDIT THIS LINE
    fileNamePrefix='file_name', ### <=== EDIT THIS LINE
    scale=10,  # Native resolution
    region=my_region,
    maxPixels=1e9,
    formatOptions={'cloudOptimized': True}
)

task.start()
print(f"Task Status: {task.status()}")
